### ingest_daily_support_tickets

In [3]:
import os
import pandas as pd
import boto3
from io import StringIO
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from urllib.parse import quote_plus

from dotenv import load_dotenv

load_dotenv("sample.env")

True

In [4]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [5]:
# ---------- CONFIG ----------
db_config = {
    "host": "localhost",
    "port": "3306",
    "user": "root",  # change
    "password": quote_plus("Myself@15"), # change
    "database": "careplus_support_db"
}

S3_BUCKET = "amzn-careplus-data-store" 
S3_PREFIX = "support-tickets/raw/"  
DATE_TRACKER_FILE = "date_tracker.txt"

import os

AWS_CONFIG = {
    "aws_access_key_id": os.getenv("AWS_ACCESS_KEY"),
    "aws_secret_access_key": os.getenv("SECRET_KEY"),
    "region_name": os.getenv("REGION")
}

In [6]:
# ---------- UTILITY FUNCTIONS ----------
def get_engine(config):
    return create_engine(f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}")

def upload_to_s3(df, bucket, key):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)

    s3 = boto3.client('s3', **AWS_CONFIG)
    s3.put_object(Bucket=bucket, Key=key, Body=csv_buffer.getvalue())
    print(f"✅ Uploaded to s3://{bucket}/{key}")

def read_last_date(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r',encoding='utf-8') as f:
            return f.read().strip()
    return "2025-06-30"  # Starting point before 1st July

def update_last_date(file_path, new_date):
    with open(file_path, 'w') as f:
        f.write(new_date)

def get_next_date(last_date_str):
    last_date = datetime.strptime(last_date_str, "%Y-%m-%d")
    next_date = last_date + timedelta(days=1)
    return next_date.strftime("%Y-%m-%d")

# ---------- MAIN INGESTION LOGIC ----------
def run_ingestion():
    engine = get_engine(db_config)
    last_date = read_last_date(DATE_TRACKER_FILE)
    next_date = get_next_date(last_date)

    # Query only that day’s data
    query = f"""
        SELECT * FROM support_tickets
        WHERE DATE(created_at) = '{next_date}';
    """
    df = pd.read_sql(query, engine)
    print(df.shape)
    print(df.head())

    if df.empty:
        print(f"⚠️ No data found for {next_date}. Skipping upload.")
        return

    # Upload to S3
    s3_key = f"{S3_PREFIX}support_tickets_{next_date}.csv"
    upload_to_s3(df, S3_BUCKET, s3_key)

    # Update date tracker
    update_last_date(DATE_TRACKER_FILE, next_date)
    print(f"📅 Updated tracker to {next_date}")

# Run
if __name__ == "__main__":
    run_ingestion()

(33, 12)
    ticket_id        created_at       resolved_at   agent priority  \
0  TCK0712000  2025-07-12 00:37  2025-07-12 10:10   Sneha      Low   
1  TCK0712001  2025-07-12 00:42  2025-07-12 14:00   Rohit   Medium   
2  TCK0712002  2025-07-12 00:59  2025-07-13 05:13  Ananya      Hgh   
3  TCK0712003  2025-07-12 01:11  2025-07-12 02:52  Ananya    Medum   
4  TCK0712004  2025-07-12 01:34  2025-07-12 13:34  Ananya   Medium   

  num_interactions         IssUeCat   channel    status agent_feedback  \
0                8       Bug Report     Phone  Resolved                  
1          -999999   Account Locked     Phone  Resolved                  
2                8       Bug Report     Phone  Resolved                  
3                4       Bug Report  Web Form  Resolved                  
4                7  Feature Request  Web Form  Resolved                  

  is_change Manager  
0      None    None  
1      None    None  
2      None    None  
3      None    None  
4      None    